# Prophet

In [5]:
# Imports
import pandas as pd
from fbprophet import Prophet
import numpy as np
from sklearn.metrics import r2_score

In [3]:
# Carregar os dados de treinamento
train_data = pd.read_excel('../baseDeDados/dataBaseTratada.xlsx')

# Carregar os dados de teste
test_data = pd.read_excel('../baseDeDados/dataBaseTeste.xlsx')

In [4]:
# Preparar os dados de treinamento para Prophet
train_prophet_data = train_data[['order_purchase_timestamp', 'delivery_time']].copy()
train_prophet_data.rename(columns={'order_purchase_timestamp': 'ds', 'delivery_time': 'y'}, inplace=True)
train_prophet_data.dropna(inplace=True)

# Aplicar a transformação logarítmica aos valores de 'delivery_time'
train_prophet_data['y'] = np.log1p(train_prophet_data['y'])

# Definir limites de cap e floor para que o modelo tenda a um valor próximo de 13
train_prophet_data['cap'] = np.log1p(13.5)  # valor um pouco maior que 13
train_prophet_data['floor'] = np.log1p(12.5)  # valor um pouco menor que 13

# Criar e treinar o modelo Prophet com sazonalidades adicionais e ajustes de hiperparâmetros
model = Prophet(
    growth='logistic',
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative',
    changepoint_prior_scale=0.1,
    seasonality_prior_scale=10.0
)

# Adicionar sazonalidades personalizadas (se aplicável)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Treinar o modelo
model.fit(train_prophet_data)

# Fazer previsões nos dados de treinamento
forecast = model.predict(train_prophet_data)

# Aplicar a transformação inversa (exponencial) aos valores previstos
forecast['yhat'] = np.expm1(forecast['yhat'])

# Preparar os dados de teste para Prophet
test_prophet_data = test_data[['order_purchase_timestamp']].copy()
test_prophet_data.rename(columns={'order_purchase_timestamp': 'ds'}, inplace=True)

# Definir cap e floor para os dados de teste
test_prophet_data['cap'] = np.log1p(13.5)
test_prophet_data['floor'] = np.log1p(12.5)

# Fazer previsões nos dados de teste
forecast_test = model.predict(test_prophet_data)

# Aplicar a transformação inversa (exponencial) aos valores previstos
forecast_test['yhat'] = np.expm1(forecast_test['yhat'])

# Adicionar previsões ao dataset de teste
test_data['previsoes'] = forecast_test['yhat']

# Exibir o resultado
test_data[['order_purchase_timestamp', 'previsoes']]

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


KeyboardInterrupt: 

In [7]:
# Exibir o resultado
result = test_data[['previsoes']]
result.reset_index(inplace=True)
result[['order_purchase_timestamp', 'previsoes']]

,order_purchase_timestamp,previsoes
0,2017-11-25 23:51:00,NaN
1,2017-11-29 15:10:00,NaN
2,2018-01-03 09:44:00,NaN
3,2018-01-12 15:38:00,NaN
4,2018-01-17 10:50:00,NaN
...,...,...
28147,2018-08-29 12:25:00,NaN
28148,2018-08-29 14:18:00,NaN
28149,2018-08-29 14:18:00,NaN
28150,2018-08-29 14:52:00,NaN


In [8]:
# Avaliação do modelo
mse = mean_squared_error(train_data['delivery_time'], train_data['delivery_time_pred'])
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 68.17652877984943


In [20]:
# Mostrar os resultados
print(test_data[['order_id', 'previsoes']].head())

                           order_id  previsoes
0  cee3292f46ede6ea1dfabfcb200fcf47  12.797736
1  50dca53ca33b739bef09e7933e8b380e  13.258393
2  8087ec71e393d4dc6fc48041fe63cd51  11.128320
3  e6b6557ce111de79b31cc857f20ba212  13.072897
4  0b09c5e4c2512f627190ac55a78c35a3  11.359839


In [21]:
# Alinhar os índices dos dados reais e previstos para garantir o mesmo tamanho
aligned_data = pd.merge(train_prophet_data, forecast[['ds', 'yhat']], on='ds', how='inner')

# Calcular o R^2 usando os valores reais e previstos
r2 = r2_score(aligned_data['y'], aligned_data['yhat'])
print(f'R^2: {r2}')

R^2: -310.640917977477


In [9]:
# Enviando para um dataFrame do Kaggle
dfKaggle = result[['order_purchase_timestamp', 'previsoes']]

# Salvar os resultados
dfKaggle.to_csv('../resultadoPrevisoes/previsaoProphet.csv', index=False)